In [19]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 8.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [20]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [21]:
with open('dataset/toxic_comments.txt', "r") as file:
    file_readen = file.read().split("\n")

labels = np.array(list(map(lambda x: list(map(lambda y: y[9 :], x.split(" ")[0].split(","))), file_readen))[: -1], dtype="object")
text = list(map(lambda x: " ".join(x.split(" ")[1 :]), file_readen))[: -1]
df = pd.DataFrame(np.array([text, labels]).T, columns = ["text", "label"])
print(df)

                                                     text     label
0                                    скотина! что сказать  [INSULT]
1       я сегодня проезжала по рабочей и между домами ...  [NORMAL]
2       очередной лохотрон. зачем придумывать очередно...  [NORMAL]
3       ретро дежавю ... сложно понять чужое сердце , ...  [NORMAL]
4                 а когда мы статус агрогородка получили?  [NORMAL]
...                                                   ...       ...
248285                       правильно всё по пять (5)...  [NORMAL]
248286  ёбанные нубы заходите на сервер мой ник _creep...  [INSULT]
248287  а у меня наверное рекорд в 1962 году в училище...  [NORMAL]
248288                              спасибо всем большое)  [NORMAL]
248289  нельзя ли увеличить хотя бы в два раза некотор...  [NORMAL]

[248290 rows x 2 columns]


In [22]:
df = df.sample(frac=0.5)
print(df)

                                                     text             label
147438                            вечная память погранцу!          [NORMAL]
30156                номер не подскажите, как записаться?          [NORMAL]
118427  лукашенко за себя переживает. ​россию возможно...          [NORMAL]
246481                   дай боженька тебе здоровья данил          [NORMAL]
25537   ну вот вы хоть бы смотрели что за бред пишете-...          [NORMAL]
...                                                   ...               ...
154244                  из двух яиц. 2-фото - с куркумой.          [NORMAL]
157376  дебил блин к яйцам своим прислони и на курок н...          [INSULT]
181093  вот на сурка ездили сравнивали коробки !и в ко...          [INSULT]
176769             назвать его дебилом -дебила оскорбить!          [INSULT]
215737                                 разорвать ублюдка!  [INSULT, THREAT]

[124145 rows x 2 columns]


In [23]:
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]
print(data[:10])

[('вечная память погранцу!', ['NORMAL']), ('номер не подскажите, как записаться?', ['NORMAL']), ('лукашенко за себя переживает. \u200bроссию возможно хочет припугнуть и американцам понравиться.', ['NORMAL']), ('дай боженька тебе здоровья данил', ['NORMAL']), ('ну вот вы хоть бы смотрели что за бред пишете-ну россия братский народ а украина раве не братский народ то!!!и уже пора прекратить этот визг про бандеровцев-подустали все уже от вашей брехни и писанины!!!правильно делает-лукаш-не отдает на ратерзание беларусь ворам московским-везде своих в конце концов хватает-а тут беспредельщики отпетые-уголощина-россию всю разграбили и туда добраться хотят-беларусь живут небогато но там порядок а не бнспредельный грабеж-как в россии кто во что горазд', ['NORMAL']), ('лучше бы набережную зделали...', ['NORMAL']), ('21 см больше ширина 8 см если чо обращайся', ['NORMAL']), ('а ей толстый в рот чтоб меньше пиздела', ['INSULT', 'OBSCENITY']), ('вы что делаете сволочи, убивать вас надо!', ['INSULT'

In [24]:
train_data, valid_data = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)
valid_data, test_data = train_test_split(valid_data, test_size=0.1, random_state=13, shuffle=True)

In [25]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, labels in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        doc.cats = {"normal": 0,
                    "insult": 0,
                    "threat": 0,
                    "obscenity": 0}
        if 'NORMAL' in labels:
          doc.cats["normal"] = 1
        if 'INSULT' in labels:
          doc.cats["insult"] = 1
        if 'THREAT' in labels:
          doc.cats["threat"] = 1
        if 'OBSCENITY' in labels:
          doc.cats["obscenity"] = 1
        docs.append(doc)
    return docs

In [26]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

100%|██████████| 5000/5000 [00:38<00:00, 131.22it/s]


In [27]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("spacy/train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("spacy/valid.spacy")

100%|██████████| 5000/5000 [00:37<00:00, 134.08it/s]


In [28]:
! python -m spacy init fill-config base_config_cnn.cfg config_cnn.cfg

✔ Auto-filled config with all values
✔ Saved config
config_cnn.cfg
You can now add your data and train your pipeline:
python -m spacy train config_cnn.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [29]:
! python -m spacy train config_cnn.cfg --output ./output --paths.train ./spacy/train.spacy --paths.dev ./spacy/valid.spacy

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.36       52.22    0.52
  0     200          17.78       65.68    0.66
  0     400          17.67       75.08    0.75
  0     600          13.44       79.29    0.79
  1     800           9.77       83.46    0.83
  2    1000           9.31       83.86    0.84
  2    1200           5.80       83.90    0.84
  3    1400           4.12       84.34    0.84
  4    1600           2.73       86.96    0.87
  6    1800           1.92       87.76    0.88
  7    2000           1.17       86.94    0.87
  9    2200           0.79       88.24    0.88
 12    2400           0.55       87.97    0.88
 1

In [38]:
import spacy
# load the best model from training
nlp = spacy.load("output/model-best")    
for text, labels in test_data[:20]:
    doc = nlp(text)
    print(f"Sample: {text}\nLabels: {labels}\nOutput: {doc.cats}\n")

3725
Sample: вот и твоя лампочка дешовая сможет и током убить
Labels: ['NORMAL']
Output: {'normal': 0.003944293595850468, 'insult': 0.9712647795677185, 'threat': 0.5361783504486084, 'obscenity': 0.0014036307111382484}

Sample: ну все тоже самое, пишет загрузка и все.
Labels: ['NORMAL']
Output: {'normal': 0.9972240924835205, 'insult': 0.0022688533645123243, 'threat': 0.00011853757314383984, 'obscenity': 1.9444718418526463e-05}

Sample: помним скорбим .0
Labels: ['NORMAL']
Output: {'normal': 0.9999943971633911, 'insult': 1.886250402094447e-06, 'threat': 8.815339242573828e-06, 'obscenity': 1.8113003534381278e-05}

Sample: так сколько мне оплатить то?
Labels: ['NORMAL']
Output: {'normal': 0.9999974966049194, 'insult': 1.0838529078682768e-06, 'threat': 1.4471313988906331e-05, 'obscenity': 1.3646133993461262e-05}

Sample: скопление пидорасов,усов не хватает.
Labels: ['INSULT']
Output: {'normal': 0.0008266202639788389, 'insult': 0.9995357990264893, 'threat': 0.01228500995784998, 'obscenity': 